In [3]:
import numpy as np

In [47]:
def simple_conv2d(input_matrix: np.ndarray, kernel: np.ndarray, padding: int, stride: int):
	input_height, input_width = input_matrix.shape
	kernel_height, kernel_width = kernel.shape

	x = input_matrix

	if padding > 0:
		x = np.concatenate([np.zeros([len(x), padding]), x, np.zeros([len(x), padding])], axis=1)
		x = np.concatenate([np.zeros([padding, len(x[0])]), x, np.zeros([padding, len(x[0])])], axis=0)
	
	x = x[:, :, None]
	ys = []
	for i in range(len(kernel)):
		ys.append(x[i: len(x) + i + 1 - len(kernel)])
	x = np.concatenate(ys, axis=-1)

	x = x[:, :, :, None]
	ys = []
	for i in range(len(kernel[0])):
		ys.append(x[:, i: len(x[0]) + i + 1 - len(kernel)])
	x = np.concatenate(ys, axis=-1)

	if stride > 1:
		x = x[::stride, ::stride, :, :]
	print(x.shape)

	x = np.einsum("ABCD,CD->AB", x, kernel)

	return x


In [48]:
input_matrix = np.array([
    [1., 2., 3., 4., 5.],
    [6., 7., 8., 9., 10.],
    [11., 12., 13., 14., 15.],
    [16., 17., 18., 19., 20.],
    [21., 22., 23., 24., 25.],
])
kernel = np.array([
    [1., 2.],
    [3., -1.],
])
padding, stride = 0, 1
expected = np.array([
    [ 16., 21., 26., 31.],
    [ 41., 46., 51., 56.],
    [ 66., 71., 76., 81.],
    [ 91., 96., 101., 106.],
])
output = simple_conv2d(input_matrix, kernel, padding, stride)
print(output)

(4, 4, 2, 2)
[[ 16.  21.  26.  31.]
 [ 41.  46.  51.  56.]
 [ 66.  71.  76.  81.]
 [ 91.  96. 101. 106.]]


In [49]:
# Test 2
input_matrix = np.array([
    [1., 2., 3., 4., 5.],
    [6., 7., 8., 9., 10.],
    [11., 12., 13., 14., 15.],
    [16., 17., 18., 19., 20.],
    [21., 22., 23., 24., 25.],
])
kernel = np.array([
    [.5, 3.2],
    [1., -1.],
])
padding, stride = 2, 2
expected = np.array([
        [ -1., 1., 3., 5., 7., 15.],
        [ -4., 16., 21., 26., 31., 35.],
        [  1., 41., 46., 51., 56., 55.],
        [  6., 66., 71., 76., 81., 75.],
        [ 11., 91., 96., 101., 106., 95.],
        [ 42., 65., 68., 71., 74.,  25.],
    ])
output = simple_conv2d(input_matrix, kernel, padding, stride)
print(output)

(4, 4, 2, 2)
[[ 0.   0.   0.   0. ]
 [ 0.   5.9 13.3 12.5]
 [ 0.  42.9 50.3 27.5]
 [ 0.  80.9 88.3 12.5]]


In [43]:
# Provided solution
def simple_conv2d(input_matrix: np.ndarray, kernel: np.ndarray, padding: int, stride: int):
    input_height, input_width = input_matrix.shape
    kernel_height, kernel_width = kernel.shape

    padded_input = np.pad(input_matrix, ((padding, padding), (padding, padding)), mode='constant')
    input_height_padded, input_width_padded = padded_input.shape

    output_height = (input_height_padded - kernel_height) // stride + 1
    output_width = (input_width_padded - kernel_width) // stride + 1

    output_matrix = np.zeros((output_height, output_width))

    for i in range(output_height):
        for j in range(output_width):
            region = padded_input[i*stride:i*stride + kernel_height, j*stride:j*stride + kernel_width]
            output_matrix[i, j] = np.sum(region * kernel)

    return output_matrix